# Streptococcus comparative genomics

Comparing protein domain presence when a specific doamin such as LysM is present vs when it's not.

In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from Bio import Phylo

cwd = os.getcwd()
if cwd.endswith('notebook'):
    os.chdir('..')
    cwd = os.getcwd()

from src.cell_wall_binding_domains import cwb_domains

In [ ]:
sns.set_theme(palette='colorblind', font_scale=1.3)
palette_colorblind = sns.color_palette('colorblind').as_hex()
palette_pastel = sns.color_palette('pastel').as_hex()

data_folder = Path('./data/')
assert data_folder.is_dir()

db_proka = Path('../db_proka/')
assert db_proka.is_dir()

gtdb_folder = Path('../data/gtdb_r220/')
assert gtdb_folder.is_dir()

strep_folder = gtdb_folder / 'Streptococcus'
assert strep_folder.is_dir()

## Load data

In [3]:
metadata_df = pd.read_csv(strep_folder / 'genomes_metadata.csv', index_col='assembly_accession')
metadata_df.head()

,accession,ambiguous_bases,checkm2_completeness,checkm2_contamination,checkm2_model,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,...,trna_aa_count,trna_count,trna_selenocysteine_count,domain,gtdb_phylum,gtdb_class,gtdb_order,gtdb_family,gtdb_genus,gtdb_species
assembly_accession,,,,,,,,,,,,,,,,,,,,,
GCF_900636555.1,RS_GCF_900636555.1,0,100.00,0.14,Specific,100.00,0.00,475,o__Lactobacillales (UID544),267,...,19,59,0,Bacteria,Bacillota,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus gordonii
GCF_015908985.1,RS_GCF_015908985.1,0,100.00,0.23,Specific,100.00,0.00,475,o__Lactobacillales (UID544),267,...,19,56,0,Bacteria,Bacillota,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus suis
GCF_001266635.1,RS_GCF_001266635.1,0,100.00,0.06,Specific,99.82,0.00,524,f__Streptococcaceae (UID545),282,...,18,63,0,Bacteria,Bacillota,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus agalactiae
GCF_004154025.1,RS_GCF_004154025.1,0,99.99,0.13,Specific,99.85,0.00,676,g__Streptococcus (UID722),182,...,19,67,0,Bacteria,Bacillota,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus pyogenes
GCF_029011635.1,RS_GCF_029011635.1,0,100.00,0.20,Specific,100.00,0.18,524,f__Streptococcaceae (UID545),282,...,19,80,0,Bacteria,Bacillota,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus agalactiae


In [72]:
pfam_df = pd.read_csv(strep_folder / 'Streptococcus_all_proteins.pfam.csv', index_col='assembly_accession')
pfam_df['gtdb_species'] = [metadata_df.loc[a, 'gtdb_species'] for a in pfam_df.index]

tigr_df = pd.read_csv(strep_folder / 'Streptococcus_all_proteins.tigr.csv', index_col='assembly_accession')
tigr_df['gtdb_species'] = [metadata_df.loc[a, 'gtdb_species'] for a in tigr_df.index]

## How many LysM per genomes?

In [82]:
lysM_df = pfam_df[pfam_df['hmm_query'] == 'LysM'].copy()

n_copies_per_species = lysM_df.reset_index()[
    ['gtdb_species', 'id', 'assembly_accession']
].groupby('gtdb_species').nunique()

n_copies_per_species['ratio'] = n_copies_per_species['id'] / n_copies_per_species['assembly_accession']
n_copies_per_species.sort_values('assembly_accession', ascending=False).head(10)

,id,assembly_accession,ratio
gtdb_species,,,
Streptococcus pyogenes,271,271,1.000000
Streptococcus agalactiae,611,197,3.101523
Streptococcus pneumoniae,313,158,1.981013
Streptococcus suis,268,109,2.458716
Streptococcus thermophilus,310,88,3.522727
Streptococcus dysgalactiae,57,43,1.325581
Streptococcus mutans,68,26,2.615385
Streptococcus iniae,14,14,1.000000
Streptococcus suis_W,39,13,3.000000


In [85]:
s_pyogenes_repr = 'GCF_002055535.1'
df = pfam_df.loc[s_pyogenes_repr]
df[df['hmm_query'] == 'LysM']

,id,protein_id,hmm_accession,hmm_query,evalue,bitscore,accuracy,start,end,gtdb_species
assembly_accession,,,,,,,,,,
GCF_002055535.1,WP_010921965.1@GCF_002055535.1,WP_010921965.1,PF01476.25,LysM,1.600000e-10,37.6,0.97,48,91,Streptococcus pyogenes


## LysM in S. equi

We spotted a pattern of differential presence of LysM in certain strains of _Steptococcus equi_.

In [44]:
accessions_s_equi = metadata_df[
    metadata_df['gtdb_species'] == 'Streptococcus equi'
].index

pfam_s_equi = pfam_df.loc[accessions_s_equi]
tigr_s_equi = tigr_df.loc[accessions_s_equi]

accessions_yes_LysM = set(pfam_s_equi[
    pfam_s_equi['hmm_query'] == 'LysM'
].index)

accessions_no_LysM = set(accessions_s_equi) - accessions_yes_LysM

print(f'Number of S. equi genomes:              {len(accessions_s_equi):,}')
print(f'Number of S. equi genones with LysM:    {len(accessions_yes_LysM):,}')
print(f'Number of S. equi genones without LysM: {len(accessions_no_LysM):,}')

Number of S. equi genomes:              38
Number of S. equi genones with LysM:    4
Number of S. equi genones without LysM: 34


In [45]:
extra_in_no = (
    (
        set(pfam_s_equi.loc[sorted(accessions_no_LysM)]['hmm_query'].values) |
        set(tigr_s_equi.loc[sorted(accessions_no_LysM)]['hmm_query'].values)
    ) - 
    (
        set(pfam_s_equi.loc[sorted(accessions_yes_LysM)]['hmm_query'].values) |
        set(tigr_s_equi.loc[sorted(accessions_yes_LysM)]['hmm_query'].values)
    )
)
len(extra_in_no)

263

In [46]:
pfam_no_lysM = pfam_s_equi.loc[sorted(accessions_no_LysM)]
tigr_no_lysM = tigr_s_equi.loc[sorted(accessions_no_LysM)]

domain_with_count = []
for domain in extra_in_no:
    if 'TIGR' in domain:
        df = tigr_no_lysM
    else:
        df = pfam_no_lysM

    count = len(set(df[df['hmm_query'] == domain].index))
    percent = float(np.round(100 * count / len(accessions_no_LysM), 1))

    domain_with_count.append((domain, count, percent))

domain_with_count = sorted(domain_with_count, key=lambda t: t[1], reverse=True)
domain_with_count[:20]

[('TIGR03426', 26, 76.5),
 ('Malt_amylase_C', 24, 70.6),
 ('AAA_10', 21, 61.8),
 ('FokI_D3', 21, 61.8),
 ('FokI_D1', 21, 61.8),
 ('FokI_dom_2', 21, 61.8),
 ('FokI_cleav_dom', 21, 61.8),
 ('MethyltransfD12', 20, 58.8),
 ('Polysacc_synt_3', 20, 58.8),
 ('TIGR00571', 19, 55.9),
 ('Phage_int_SAM_5', 19, 55.9),
 ('Gp58', 18, 52.9),
 ('Cellulase', 17, 50.0),
 ('RloB', 16, 47.1),
 ('T6_Ig_like', 16, 47.1),
 ('HTH_36', 16, 47.1),
 ('TIGR01637', 15, 44.1),
 ('TIGR01451', 15, 44.1),
 ('DUF1617', 14, 41.2),
 ('GA', 14, 41.2)]

In [47]:
missing_in_no = (
    (
        set(pfam_s_equi.loc[sorted(accessions_yes_LysM)]['hmm_query'].values) |
        set(tigr_s_equi.loc[sorted(accessions_yes_LysM)]['hmm_query'].values)
    ) - 
    (
        set(pfam_s_equi.loc[sorted(accessions_no_LysM)]['hmm_query'].values) |
        set(tigr_s_equi.loc[sorted(accessions_no_LysM)]['hmm_query'].values)
    )
)
len(missing_in_no)

16

In [48]:
pfam_yes_lysM = pfam_s_equi.loc[sorted(accessions_yes_LysM)]
tigr_yes_lysM = tigr_s_equi.loc[sorted(accessions_yes_LysM)]

missing_domain_with_count = []
for domain in missing_in_no:
    if 'TIGR' in domain:
        df = tigr_yes_lysM
    else:
        df = pfam_yes_lysM

    count = len(set(df[df['hmm_query'] == domain].index))
    percent = float(np.round(100 * count / len(accessions_yes_LysM), 1))

    missing_domain_with_count.append((domain, count, percent))

missing_domain_with_count = sorted(missing_domain_with_count, key=lambda t: t[1], reverse=True)
missing_domain_with_count[:20]

[('DUF6900', 4, 100.0),
 ('Amidase_3', 4, 100.0),
 ('LysM', 4, 100.0),
 ('DUF2326', 3, 75.0),
 ('MC7', 3, 75.0),
 ('CTD11', 3, 75.0),
 ('PhdYeFM_antitox', 3, 75.0),
 ('ToxN_toxin', 3, 75.0),
 ('SLFN-g3_helicase', 3, 75.0),
 ('MeaB', 3, 75.0),
 ('DEDD_Tnp_IS110', 2, 50.0),
 ('HNH_3', 1, 25.0),
 ('Imm63', 1, 25.0),
 ('DUF4145', 1, 25.0),
 ('DUF3781', 1, 25.0),
 ('TrbL', 1, 25.0)]

Not particularly conclusive at this level of analysis.

## Cell wall HMM search

HMM models from [Megrian et al., 2022](https://doi.org/10.1038/s41564-022-01257-y).

In [60]:
cw_df = pd.read_csv(strep_folder / 'Streptococcus_all_proteins.cell_wall.csv', index_col='assembly_accession')

# Keep only one model per protein – best scoring one.
cw_df = cw_df.sort_values(['id', 'bitscore'], ascending=[True, False]).drop_duplicates(['id'])

cw_df.head()

,id,protein_id,hmm_accession,hmm_query,evalue,bitscore,accuracy,start,end
assembly_accession,,,,,,,,,
GCA_000013525.1,ABF35082.1@GCA_000013525.1,ABF35082.1,newDdlB,newDdlB,6.500000e-08,28.6,0.77,126,303
GCA_000013525.1,ABF35084.1@GCA_000013525.1,ABF35084.1,newDdlB,newDdlB,3.500000e-11,39.3,0.79,106,289
GCA_000013525.1,ABF35137.1@GCA_000013525.1,ABF35137.1,newFtsI,newFtsI,1.900000e-21,73.0,0.79,329,642
GCA_000013525.1,ABF35311.1@GCA_000013525.1,ABF35311.1,newMraY,newMraY,1.400000e-35,119.7,0.79,0,330
GCA_000013525.1,ABF35345.1@GCA_000013525.1,ABF35345.1,newMurD,newMurD,3.800000e-05,19.8,0.77,0,117


In [66]:
cw_df_yes_lysM = cw_df.loc[sorted(accessions_yes_LysM)]

cw_yes_lysM_grouped = cw_df_yes_lysM[
    ['id', 'hmm_query']
].groupby(
    'hmm_query'
).nunique().sort_values('id', ascending=False).rename(
    columns={
        'id': 'count',
    }
)
cw_yes_lysM_grouped['ratio'] = cw_yes_lysM_grouped['count'] / len(accessions_yes_LysM)
cw_yes_lysM_grouped

,count,ratio
hmm_query,,
newDdlB,20,5.0
newFtsI,20,5.0
newMraW,12,3.0
newMurA,12,3.0
newMurD,12,3.0
newFtsA,8,2.0
newFtsW,8,2.0
newMraY,8,2.0
newMurE,8,2.0


In [67]:
cw_df_no_lysM = cw_df.loc[sorted(accessions_no_LysM)]

cw_no_lysM_grouped = cw_df_no_lysM[
    ['id', 'hmm_query']
].groupby(
    'hmm_query'
).nunique().sort_values('id', ascending=False).rename(
    columns={
        'id': 'count',
    }
)
cw_no_lysM_grouped['ratio'] = cw_no_lysM_grouped['count'] / len(accessions_no_LysM)
cw_no_lysM_grouped

,count,ratio
hmm_query,,
newFtsI,170,5.000000
newDdlB,169,4.970588
newMurA,102,3.000000
newMurD,101,2.970588
newMraW,94,2.764706
newFtsW,68,2.000000
newMraY,68,2.000000
newFtsA,67,1.970588
newMurC,52,1.529412


In [69]:
set(cw_no_lysM_grouped.index) - set(cw_yes_lysM_grouped.index)

{'newMraZ'}

In [70]:
set(cw_yes_lysM_grouped.index) - set(cw_no_lysM_grouped.index)

set()